If not yet available some libraries and their python bindings have to be installed :<br>
- gmsh (best installed globally through package management system)
- python3 -m pip install pygmsh --user
- VTK (best installed globally through package management system)
- python3 -m pip install vtk --user

In [1]:
import numpy as np
from scipy import constants
import pygmsh
from MeshedFields import *

# Create a meshed screen with a central hole

The screen is rectangular (215*150 mm) with a 4mm central hole.<br>
Typical cell size is 5 mm along the outside of the screen and 1.0 mm near the inner hole.<br>
The wavelength is 0.3 mm only. That means we need about 0.02 mm resolution along the x/z axis.<br>
When generating the geometry we stretch it by a factor 50 and undo that after creating the mesh,
effectively generating cells that are denser along the the stretched direction.

In [2]:
with pygmsh.geo.Geometry() as geom:
    
    Lx = 0.215
    Ly = 0.150
    Ri = 0.002
    lca = 0.005
    lci = 0.001
    stretch = 50.0

    p1 = geom.add_point([Lx/2.0*stretch, Ly/2.0], lca)
    p2 = geom.add_point([-Lx/2.0*stretch, Ly/2.0], lca)
    p3 = geom.add_point([-Lx/2.0*stretch, -Ly/2.0], lca)
    p4 = geom.add_point([Lx/2.0*stretch, -Ly/2.0], lca)
    p1i = geom.add_point([Ri*stretch, 0.0], lci)
    p2i = geom.add_point([0.0, Ri], lci)
    p3i = geom.add_point([-Ri*stretch, 0.0], lci)
    p4i = geom.add_point([0.0, -Ri], lci)
    pc = geom.add_point([0.0, 0.0])
    pa = geom.add_point([0.0, 0.01])

    # the mesh is circumscribed with a polygon
    l1 = geom.add_line(p1, p2)
    l2 = geom.add_line(p2, p3)
    l3 = geom.add_line(p3, p4)
    l4 = geom.add_line(p4, p1)
    outline = geom.add_curve_loop([l1, l2, l3, l4])

    # the hole is circumscribed with four elliptic arcs
    e1i = geom.add_ellipse_arc(start=p1i, center=pc, point_on_major_axis=pa, end=p2i)
    e2i = geom.add_ellipse_arc(start=p2i, center=pc, point_on_major_axis=pa, end=p3i)
    e3i = geom.add_ellipse_arc(start=p3i, center=pc, point_on_major_axis=pa, end=p4i)
    e4i = geom.add_ellipse_arc(start=p4i, center=pc, point_on_major_axis=pa, end=p1i)
    hole = geom.add_curve_loop([e1i,e2i,e3i,e4i])

    pl = geom.add_plane_surface(outline, holes=[hole])
    
    mesh = geom.generate_mesh()

In [3]:
mesh

<meshio mesh object>
  Number of points: 85036
  Number of cells:
    line: 4764
    triangle: 165304
    vertex: 10

In [4]:
# un-stretch
pts = np.array([np.array([p[0]/stretch,p[1],0.0]) for p in mesh.points])
tris = mesh.cells_dict['triangle']

The z-position of all mesh points is computed to lay on a toroid with 1.625 m focal length.<br>
The radius of curvature in the plane is 2 f, out of plane f.<br>
The elevation in z direction is computed for x and y positions independently, assuming the size of the mirror being small in comparison to its focal length.

In [5]:
def ToroidZ(x,y,f):
    # return f-math.sqrt(f*f-y*y) + 2*f-math.sqrt(4*f*f-x*x)
    return math.sqrt( math.pow(math.sqrt(f*f-y*y)+f,2) -x*x ) - 2*f
pts = np.array([np.array([p[0],p[1],ToroidZ(p[0],p[1],1.625)]) for p in pts])

In [6]:
screen = MeshedField(pts,tris)
print("%d points" % len(screen.points))
print("%d triangles" % len(screen.triangles))
area = screen.MeshArea()
normals = screen.MeshNormals()
average = np.sum(normals, axis=0)/screen.Np
print("total mesh area = %7.3f cm²" % (1.0e4*np.sum(area)))
print("screen normal = %s" % average)

85036 points
165304 triangles
total mesh area = 322.558 cm²
screen normal = [ 5.94418597e-05 -4.01710693e-05 -9.99464687e-01]


In [7]:
screen.ShowMeshedField(showAxes=True)

The screen is placed at z=3.625 m from th origin. A beam is assumed to propagate in z direction
The fields shall be reflected to th x direction. The screen normal is pointing in the negative z and positive x direction (x - left, y - up). To achieve that the screen has to be rotated by 45 degrees about the y axis.

In [8]:
def RotXZ(φ):
    return np.array([[np.cos(φ),0,-np.sin(φ)],[0,1,0],[np.sin(φ),0,np.cos(φ)]])
RR = RotXZ(45.0/180.0*math.pi)
pts = np.array([np.dot(RR,p) for p in pts])

In [9]:
screen = MeshedField(pts,tris)
print("%d points" % len(screen.points))
print("%d triangles" % len(screen.triangles))
area = screen.MeshArea()
normals = screen.MeshNormals()
average = np.sum(normals, axis=0)/screen.Np
print("total mesh area = %7.3f cm²" % (1.0e4*np.sum(area)))
print("screen normal = %s" % average)

85036 points
165304 triangles
total mesh area = 322.558 cm²
screen normal = [ 7.06770290e-01 -4.01710693e-05 -7.06686226e-01]


In [10]:
screen.ShowMeshedField(showAxes=True)

In [11]:
pts = np.array([p+np.array([0.0,0.0,3.625]) for p in pts])
screen = MeshedField(pts,tris)

In [12]:
screen.ShowMeshedField(showAxes=True)

### define the timing

The beam is assumed to start at t=0. The fields are propagating with c so the expected time a signal arrives at some screen point is z/c.


In [13]:
# time step
screen.dt = 0.5e-13
# some time shift of the waveform start
delay = 15.0e-12
# all points use the same timing grid
screen.Nt = 800
screen.t0 = np.array([p[2]/constants.c-screen.Nt/2*screen.dt+delay for p in screen.pos])

In [14]:
filename="OL8_ToroidalMirrorWithHole.h5"
screen.WriteMeshedField(filename)

not writing ElMagField dataset
